# Taller SPR 2023
# Métodos para Analizar Oscilaciones Neuronales y Actividad Aperiódica

La primera parte de este taller (Digital Signal Processing y Simulaciones) consiste en 2 secciones separadas en Jupyter Notebooks. El primer Notebook (el presente) demuestra como el espectro de potencia cambia dependiendo de la temporalidad de la señal simulada. En el segundo Notebook se explora los básicos de la aplicación de filtros, y cómo un filtro puede alterar tus resultados, de tal manera que puedas ser consciente de ello durante tus propios análisis. Aquí te dejamos una lista del contenido que puedes encontrar en este Notebook:

- Espectro de Potencia
    - Función Delta
    - Ruido blanco (White noise)
- Oscilaciones Sinusoidales
    - Una oscilación
    - Varias oscilaciones
    - Oscilaciones múltiples
- Oscilaciones no estacionarias: Bursting
- Oscilaciones no-senoidales
- Oscilaciones integradas en la actividad aperiódica

## Lección 1: Digital Signal Processing y Simulaciones

En este Notebook cubriremos las bases de algunas funciones para simular tus propios datos usando la biblioteca NeuroDSP (https://neurodsp-tools.github.io/neurodsp/index.html)

### Configuración

In [8]:
# Instalar los paquetes necesarios (para Colab)
!pip install neurodsp

#### Dependencias

In [3]:
# general 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal

# Voytek Lab tools
from neurodsp import spectral
from neurodsp import filt
from neurodsp import sim
from neurodsp import utils

#### Ajustes

In [4]:
# parámetros de la señal
N_SECONDS = 100 # duración de la señal
FS = 1000 # frecuencia de muestreo (sampling frequency)

In [5]:
# parámetros para las gráficas

# tamaño de fuente
mpl.rcParams['figure.titlesize'] = 18
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 10

# color
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['axes.facecolor'] = 'w'

#### Funciones

In [6]:
def plot_signal_and_power(time, signal, freq, spectrum, title='', logscale=False, xlims=None):
    '''
    Visualización de series de tiempo y su espectro de potencia correspondiente
    
    Parametros
    ----------
    time : 1D array, float
        vector de tiempo de la señal
    signal : 1D array, float
        serie de tiempo, ej. potenciales de campo locales (LFP) o electroencefalogramas (EEG)
    freq : 1D array, float
        vector de frecuencia para el espectro de potencia
    spectrum : 1D array, float
        espectro de potencia de la señal
    title : str, optional
        título para la figura. El título predeterminado es ''.
    logscale : bool, optional
        si se debe trazar el espectro como funci[on logarítmica. El valor predeterminado es False.
    xlims : 1D array (len=2), optional
        límites del eje x para la gráfica de serie de tiempo ([limite_inferior, limite_superior]).
        El rango predeterminado es None.

    '''
    # crear la figura
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=[12,4], gridspec_kw={'width_ratios': [3, 1]}, constrained_layout=True)
    fig.suptitle(title)

    # trazar la señal
    ax1.set(xlabel='tiempo (s)', ylabel='voltaje (au)', title='Serie de tiempo')
    ax1.plot(time, signal)
    if xlims:
        ax1.set_xlim(xlims)
        
    # trazar el espectrograma
    ax2.set(xlabel='frecuencia (Hz)', ylabel='potencia (au)', title='Espectro de Potencia (Power Spectral Density)')
    ax2.plot(freq, spectrum);
    ax2.set_xlim([.1,200])
    if logscale:
        ax2.set(xscale='log', yscale='log')


### Espectro de Potencia para Señales Simples:
Primero simularemos una función Dirac delta y ruido blanco.

#### Función Dirac delta

In [7]:
# simular la función dirac delta
dirac = np.zeros(utils.data.compute_nsamples(N_SECONDS, FS)) # crear un array de ceros
dirac[int(len(dirac)/2)] = 1 # ajustar el valor central a 1
time = utils.create_times(N_SECONDS, FS, start_val=-N_SECONDS/2)

# calcular el espectro de potencia
freq, psd_dirac = spectral.compute_spectrum(dirac, FS)

# graficar
plot_signal_and_power(time, dirac, freq, psd_dirac, title='Función Dirac Delta')

AttributeError: module 'neurodsp.utils.data' has no attribute 'compute_nsamples'

Podemos ver que esta sencilla función está asociada con una potencia distinta a cero en todas las frecuencias.

#### Señales de Ruido Blanco (White Noise)

In [8]:
# simular una señal de ruido blanco
signal_white = np.random.normal(loc=0, scale=1, size=utils.data.compute_nsamples(N_SECONDS, FS))
time = utils.create_times(N_SECONDS, FS)

# calcular el espectro de potencia de nuestra señal de ruido blanco
_, psd_white = spectral.compute_spectrum(signal_white, FS)

# graficar
plot_signal_and_power(time, signal_white, freq, psd_white, title='Señales de Ruido Blanco', xlims=[0,1])

Type:        module
String form: <module 'neurodsp.utils.data' from 'c:\\Users\\Andre\\anaconda3\\envs\\voytek_f1\\lib\\site-packages\\neurodsp\\utils\\data.py'>
File:        c:\users\andre\anaconda3\envs\voytek_f1\lib\site-packages\neurodsp\utils\data.py
Docstring:   Data related utility functions.


Podemos ver que las señales de ruido blanco tienen una potencia aproximadamente igual en todas las frecuencias. Al igual que en nuestra simulación previa, cada una de las bandas de frecuencia demuestra cierto nivel de potencia a pesar de que no hay oscilaciones presentes. Estas simulaciones nos ayudan a comprender que el solo observar cierto nivel de potencia en una banda de frecuencia específica no es evidencia suficiente para demostrar oscilaciones neuronales.

### Oscilaciones Sinusoidales
A continuación, simularemos señales sinusoidales y señales compuestas por oscilaciones múltiples.

#### simulación de una oscilación sinusoidal

In [ ]:
# configuración
freq_oscillation = 10 # frecuencia extrema de nuestra oscilación

In [ ]:
# simular una oscilación sinusoidal
signal_sin = sim.sim_oscillation(N_SECONDS, FS, freq_oscillation)

# calcular el espectro de potencia
_, psd_sin = spectral.compute_spectrum(signal_sin, FS)

# graficar
plot_signal_and_power(time, signal_sin, freq, psd_sin, title='Oscilación Sinusouidal', xlims=[0,1])

Podemos ver que la potencia de las ondas sinusoidales está asociada con bandas de frecuencia estrechas. Este ejemplo nos demuestra un vértice en el espectro de potencia en la frecuencia de la oscilación. Intenta cambiar el valor de la variable `freq_oscillation` y observa su efecto en el espectro de potencia.

#### simulación de señales compuestas por *varias* oscilaciones sinusoidales 

In [ ]:
# configuración
freq_oscillations = [10, 35, 60] # frecuencia extrema de nuestra oscilación

In [ ]:
# simular la señal
signal_sins = np.zeros_like(time)
for i_osc in range(len(freq_oscillations)):
    signal_sins += sim.sim_oscillation(N_SECONDS, FS, freq_oscillations[i_osc])

# calcular el espectro de potencia
_, psd_sins = spectral.compute_spectrum(signal_sins, FS)

# graficar
plot_signal_and_power(time, signal_sins, freq, psd_sins, 
    title='Señal con Varias Oscilaciones Sinusoidales', xlims=[0,1])

####  simulación de señales compuestas por oscilaciones sinusoidales *múltiples* (10)

In [ ]:
# configuración
osc_freq = np.random.rand(10)*100 # frecuencia extrema de nuestra oscilación
osc_amp = np.random.rand(10) # amplitud de nuestras oscilaciones

In [ ]:
# simular la señal
signal_sins2 = np.zeros_like(time)
for i_osc in range(len(osc_freq)):
    signal_sins2 += sim.sim_oscillation(N_SECONDS, FS, osc_freq[i_osc]) * osc_amp[i_osc]

# calcular el espectro de potencia
_, psd_sins2 = spectral.compute_spectrum(signal_sins2, FS)

# graficar
plot_signal_and_power(time, signal_sins2, freq, psd_sins2, 
    title='Señal con Múltiples Oscilaciones Sinusoidales', xlims=[0,1])


A pesar de que la señal parece ser 'compleja', podemos ver que nuestra potencia todavía está concentrada en las bandas de frecuencia estrechas, determinada por la variable `osc_freq`.

### Oscilaciones no estacionarias (Bursting)

Ahora simularemos oscilaciones no estacionarias. Las señales neuronales no son estacionarias, i.e. su contenido espectral cambia conforme el tiempo. Se ha demostrado que las oscilaciones surgen como ráfagas transitorias ('bursts'), manifestándose por décimas o centésimas de milisegundos (ver Stokes & Spaak, 2016; y Jones, 2016).

In [ ]:
# configuración
freq_oscillation = 10 # frecuencia extrema de nuestra oscilación
cycle = 'sine' # forma de nuestra onda oscilatoria (en este ejemplo usamos 'función seno'/'sine')

In [ ]:
# simular una oscilación no estacionaria
signal_bursty =  sim.sim_bursty_oscillation(n_seconds=N_SECONDS, fs=FS, freq=freq_oscillation, cycle=cycle)

# calcular el espectro de potencia
_, psd_bursty = spectral.compute_spectrum(signal_bursty, FS)

# graficar
plot_signal_and_power(time, signal_bursty, freq, psd_bursty, title="Oscilación 'Bursty'", xlims=[0,10])

### Oscilaciones No-Senoidales
A continuación vamos a simular oscilaciones no-senoidales. Se ha demostrado que los ritmos neuronales no son sinusoidales, frecuentemente mostrando una forma de diente de sierra y otras distorsiones de fase. Estas características en nuestras formas de onda son fisiológicamente significativas, pero requieren de un análisis más detallado el cual exploraremos más adelante en este taller. Dentro de la Sección 02 exploraremos el método cycle-by-cycle para estudiar las formas de onda de las oscilaciones neuronales. 

In [ ]:
# configuración
freq_oscillation = 10 # frecuencia extrema de nuestra oscilación
cycle = 'sawtooth' # forma de nuestra onda oscilatoria (en este ejemplo usamos 'diente de sierra'/'sawtooth')
width = 1 # criterio de forma para la onda diente de sierra (0: elevación rápida, 0.5: simétrica, 1: caída rápida)

In [ ]:
# simular la señal
signal_saw =  sim.sim_bursty_oscillation(n_seconds=N_SECONDS, fs=FS, freq=freq_oscillation, cycle=cycle, width=width)

# calcular el espectro de potencia
_, psd_saw = spectral.compute_spectrum(signal_saw, FS)

# graficar
plot_signal_and_power(time, signal_saw, freq, psd_saw,
    title='Bursty, Non-sinusoidal Oscillation', xlims=[0,3])

Podemos observar que la naturaleza no-senoidal de nuestra señal produce cierta armonía dentro de nuestro espectro de potencia. A pesar de que existe un ritmo único en la frecuencia determinada por `freq_oscillation`, también podemos observar picos adicionales en los múltiplos de esta frecuencia.

A continuación, trazaremos más ejemplos de ondas no-senoidales y su representación en dominios de frecuencia. Cada una de estas señales tiene la misma frecuancia y amplitud, pero diferentes formas de onda.

In [ ]:
# Simular 3 ejemplos de formas de onda y observar su representación en el dominio de frecuencia.
# Cada una de estas señales tiene la misma frecuencia y amplitud, pero diferentes formas de onda.

# señal de diente de sierra
time = np.arange(0, N_SECONDS, 1/FS)
signal_saw = signal.sawtooth(2*np.pi*freq_oscillation*time)
freq, psd_saw = spectral.compute_spectrum(signal_saw, FS)
plot_signal_and_power(time, signal_saw, freq, psd_saw,
    title='Diente de sierra', xlims=[0,1])
                      
# señal cuadrangular
signal_square = signal.square(2*np.pi*freq_oscillation*time)
_, psd_square = spectral.compute_spectrum(signal_square, FS)
plot_signal_and_power(time, signal_square, freq, psd_square,
    title='Cuadrangular', xlims=[0,1])

# señal triangular
signal_triangle = signal.sawtooth(2*np.pi*freq_oscillation*time, width=0.5)
_, psd_triangle = spectral.compute_spectrum(signal_triangle, FS)
plot_signal_and_power(time, signal_triangle, freq, psd_triangle,
    title='Triangular', xlims=[0,1])

Estas simulaciones nos dejan ver con mejor exactitud que un pico en el espectro de potencia no es necesariamente evidencia de una osilación en esa frecuencia. Cuando la transformada de Fourier se aplica a una señal no-senoidal, la representación de esta frecuencia incluye pseudocomponentes sinusoidales, representando la señal como una sinusoide compleja. Esta representación equivocada de la señal puede resultar en interpretaciones erróneas de la fisiología y los procesos cognitivos subyacentes. Además, este efecto alude a un mayor problema: las funciones basadas en sinusoides son ubicuas en el campo de la neurosciencia. La sección 02 de este taller explorará una solución a este problema: un enfoque ciclo-por-ciclo para analizar oscilaciones no-senoidales en el dominio del tiempo.

### Oscilaciones Dentro de la Actividad Aperiódica
Ahora, simularemos una oscilación dentro de la actividad aperiódica (o no-senoidal). Los datos neuronales (como los potenciales de campo locales, EEG, ECoG, y MEG) muestran un componente espectral invertido parecido a la función 1/f, en donde la potencia es inversamente proporcional a la frecuencia. Esto contrasta las señales de ruido blanco que simulamos anteriormente, donde la potencia es igual en todas las frecuencias. La actividad aperiódica es fisiológicamente relevante y debería de ser considerada al analizar oscilaciones.

In [ ]:
# configuración
sim_components = {'sim_bursty_oscillation': {'freq' : 10},
                  'sim_powerlaw': {'exponent' : -2}} # exponente aperiódico (pendiente espectral)

In [ ]:
# simular una oscilación sinusoidal dentro de la actividad aperiódica
signal_comb =  sim.sim_combined(n_seconds=N_SECONDS, fs=FS, components=sim_components, component_variances=[1,100])

# calcular el espectro de potencia
_, psd_comb = spectral.compute_spectrum(signal_comb, FS)

# graficar
plot_signal_and_power(time, signal_comb, freq, psd_comb,
    title='Oscilación y Actividad Aperiódica',
    logscale=True, xlims=[0,10])

Aquí podemos observar que la señal aperiódica contribuye a la potencia representada en todas las frecuencias, con la potencia disminuyendo mientras al frecuencia incrementa. De manera similar a las simulaciones que realizamos anteriormente, la oscilación sinusoidal contribuye a la potencia dentro de una banda de frecuencia estrecha. Es necesario cuantificar cuidadosamente estos componentes para poder diferenciar sus respectivas contribuciones a la cognición y el comportamiento.

Conclusión
----------

Este Notebook demuestra como simular  series de tiempo neurales usando NeuroDSP. Exploramos la relación entre representaciones de dominios de frecuencia y tiempo, destacando algunas advertencias que hay que tomar en cuenta al realizar análisis basados en Fourier. Además, introducimos las diferencias entre la actividad periódica y la aperiódica, la cual exploraremos a mayor profundidad en la sección 01, Parametrización Espectral; y se introdujo el concepto de forma de onda, el cual exploraremos más a fondo en la sección 02, Bycicle.